In [1]:
from factue.utils.types import ModelMode, ModelName, ModelProvider

In [8]:
x = ModelName("llama3.1:8b")
x

<ModelName.LLAMA_31_8B: 'llama3.1:8b'>

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import luigi
import pandas as pd
from factue.pipelines.clean_groundtruth import CleanGroundTruthTask
from factue.utils.vars import PROJECT_ROOT
from factue.utils.viz import disp

ModuleNotFoundError: No module named 'factue.pipelines.clean_groundtruth'

In [ ]:
sample_in = (
    PROJECT_ROOT / "data/01_preprocessed" / "dev" / "dev-eng" / "batch_0000.parquet"
)
df = pd.read_parquet(sample_in)
df

In [ ]:
t1 = CleanGroundTruthTask(
    input_path=sample_in,
    input_dir=PROJECT_ROOT / "data/01_preprocessed",
    output_dir=PROJECT_ROOT / "data/02_cleaned_ground_truth",
    force=True,
)

luigi.build([t1], local_scheduler=True)

In [ ]:
sample_out = (
    PROJECT_ROOT
    / "data/02_cleaned_ground_truth"
    / "dev"
    / "dev-eng"
    / "batch_0000.parquet"
)
df = pd.read_parquet(sample_out)
df

In [ ]:
# from langchain.chat_models import ChatOpenAI
# from langchain.schema import SystemMessage, HumanMessage
# from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
# from langchain.output_parsers import JsonOutputKeyToolsParser

# # 1. Set up the system prompt
# system_prompt = SystemMessage(
#     "You are an expert journalist and fact-checker. "
#     "You compare extracted claims with source posts to verify accuracy. "
#     "Return only structured JSON that answers five fact-checking questions. "
#     "Do not explain or output anything outside of the JSON structure."
# )

# # 2. Human prompt template with variables
# human_prompt_template = HumanMessagePromptTemplate.from_template(
#     """
# You are an expert journalist and fact-checker. Your task is to compare an extracted claim with the original post and determine whether the claim accurately reflects the source content.

# For each of the following questions, answer `true` or `false` based on your analysis. If your answer is `false`, provide a list of specific errors. If `true`, keep the list empty.

# Strictly respond in the following JSON format:

# {{
#   "questions": [
#     {{
#       "id": "meaning_preserved",
#       "answer": true | false,
#       "errors": [ "..." ]
#     }},
#     {{
#       "id": "correct_named_entities",
#       "answer": true | false,
#       "errors": [ "..." ]
#     }},
#     {{
#       "id": "correct_numbers",
#       "answer": true | false,
#       "errors": [ "..." ]
#     }},
#     {{
#       "id": "no_added_data",
#       "answer": true | false,
#       "errors": [ "..." ]
#     }},
#     {{
#       "id": "no_missing_data",
#       "answer": true | false,
#       "errors": [ "..." ]
#     }}
#   ]
# }}

# ---

# CLAIM:
# \"\"\"{claim}\"\"\"

# POST:
# \"\"\"{post}\"\"\"
# """
# )

# # 3. Build the complete prompt
# prompt = ChatPromptTemplate.from_messages([
#     system_prompt,
#     human_prompt_template
# ])

# # 4. Define the model
# # Example: If you are using LM Studio, set the correct base_url and API key (fake one)
# llm = ChatOpenAI(
#     model="local-llm",  # or "gpt-3.5-turbo" or any other
#     openai_api_base="http://localhost:1234/v1",  # LM Studio endpoint
#     openai_api_key="lm-studio",  # dummy key if local
#     temperature=0.0
# )

# # 5. Output parser
# parser = JsonOutputKeyToolsParser()

# # 6. LangChain call function
# def factcheck_claim_against_post(claim: str, post: str) -> dict:
#     messages = prompt.format_prompt(claim=claim, post=post).to_messages()
#     response = llm(messages)
#     result = parser.parse(response)
#     return result

# # 7. Example usage
# if __name__ == "__main__":
#     claim = "The president announced 5 billion dollars to rebuild Rome in 2025."
#     post = "In 2025, the mayor of Florence announced 500 million dollars for road repairs."

#     output = factcheck_claim_against_post(claim, post)
#     print(output)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from factue.methods.llm_call import validate_claim

In [ ]:
claim = """Photo shows Louis Armstrong as a child"""
post = """The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More None"""
print(validate_claim("v001", "v001", post, claim))

In [ ]:
from langchain_core.messages import HumanMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from factue.methods.prompts.check_claim import (
    check_claim_prompt_templates,
    check_claim_prompt_formating,
)

# from factue.methods.llm_langchain.lms import init_lms
from factue.methods.llm_langchain.ollama import init_ollama

# "llama3.1:8b"
# "deepseek-r1:8b"
llm = init_ollama(model="llama3.1:8b", temperature=0.0)


# Function to run on each row
def validate_claim(prompt_version, format_version, post, claim):
    human_prompt_template = HumanMessagePromptTemplate.from_template(
        check_claim_prompt_templates[prompt_version]
    )
    format = check_claim_prompt_formating[format_version]
    prompt = human_prompt_template.format(claim=claim, post=post, format=format)
    print(prompt.content)
    # ai_msg = llm.invoke([prompt])
    # return ai_msg.content.strip()


claim = """Photo shows Louis Armstrong as a child"""
post = """The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More None"""
print(validate_claim("v001", "v001", post, claim))

# remote llama

In [ ]:
from factue.methods.llm_langchain.ollama import init_ollama

# "llama3.1:8b"
# "deepseek-r1:8b"
llm = init_ollama(model="llama3.1:8b", temperature=0.0)

In [ ]:
llm.invoke(
    '''You are an expert journalist and fact-checker. Your task is to compare an extracted claim with the original post and determine whether the claim accurately reflects the source content.
For each of the following questions, answer `true` or `false` based on your analysis. If your answer is `false`, provide a list of specific errors. If `true`, keep the list empty.


Strictly respond in the following JSON format:
{{
  "questions": [
    {{
      "id": "meaning_preserved",
      "answer": true | false,
      "errors": [ "..." ]
    }},
    {{
      "id": "correct_named_entities",
      "answer": true | false,
      "errors": [ "..." ]
    }},
    {{
      "id": "correct_numbers",
      "answer": true | false,
      "errors": [ "..." ]
    }},
    {{
      "id": "no_added_data",
      "answer": true | false,
      "errors": [ "..." ]
    }},
    {{
      "id": "no_missing_data",
      "answer": true | false,
      "errors": [ "..." ]
    }}
  ]
}}

CLAIM:
"""Photo shows Louis Armstrong as a child"""

POST:
"""The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More None”””'''
)

# previous cell

In [ ]:
from langchain_core.messages import HumanMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from factue.methods.prompts.check_claim import templates_check_claim

# from factue.methods.llm_langchain.lms import init_lms
from factue.methods.llm_langchain.ollama import init_ollama

llm = init_ollama()


# Function to run on each row
def validate_claim(version, post, claim):
    human_prompt_template = HumanMessagePromptTemplate.from_template(
        templates_check_claim[version]
    )
    prompt = human_prompt_template.format(claim=claim, post=post)
    print(prompt.content)
    # ai_msg = llm.invoke([prompt])
    # return ai_msg.content.strip()


claim = """Photo shows Louis Armstrong as a child"""
post = """The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More None"""
print(validate_claim("v003", post, claim))

In [ ]:
df["validation"] = df.apply(lambda x: validate_claim(x["post"], x["gold"]), axis=1)

In [ ]:
disp(df[["post", "gold", "validation"]])

# old


In [ ]:
import json
import requests


def compare_claim_with_post(claim_text: str, original_post: str) -> dict:
    prompt = f"""
You are a fact-checking assistant.

Compare the claim with the original post, and determine if the claim is factually correct. If any information is wrong, missing, or added, return the issues categorized under these keys:
- "wrong_meaning": if the claim has a different meaning than the original post.
- "wrong_name": if people, places, or organizations are misidentified.
- "wrong_number": if numeric values (e.g. money, percentages, dates) are incorrect.
- "missing_data": if important details from the original post are not mentioned in the claim.
- "added_data": if the claim includes things not present in the original.

Return ONLY a valid JSON object like this:

{{
  "is_factually_correct": true | false,
  "errors": {{
    "wrong_meaning": [ "...", "..." ],
    "wrong_name": [ "...", "..." ],
    "wrong_number": [ "...", "..." ],
    "missing_data": [ "...", "..." ],
    "added_data": [ "...", "..." ]
  }}
}}

CLAIM:
\"\"\"{claim_text}\"\"\"

ORIGINAL POST:
\"\"\"{original_post}\"\"\"

Do not explain or include anything else — just the JSON.
"""

    response = requests.post(
        "http://localhost:1234/v1/completions",
        headers={"Content-Type": "application/json"},
        data=json.dumps(
            {"prompt": prompt, "max_tokens": 512, "temperature": 0.2, "stop": ["\n\n"]}
        ),
    )

    try:
        return json.loads(response.json()["choices"][0]["text"].strip())
    except Exception as e:
        return {
            "is_factually_correct": False,
            "errors": {
                "wrong_name": [],
                "wrong_number": [],
                "missing_data": [],
                "added_data": [],
                "parsing_error": [f"Model response parsing failed: {str(e)}"],
            },
        }

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage


system_prompt = SystemMessage(
    "You are an expert journalist that checks for a most concise formulation of possibly false claims extracted from the post. "
    "Compare the claim with the post and check if the claim is accurately reflecting the post and revise if needed"
    "Output only the claim. Do not add any additional information. Do not correct the false claim. The claim should be a single sentence. If there is no claim, say 'No claim'."
)


# Function to run on each row
def validate_claim(post, claim):
    text = f"Post: {post}\n\n Claim: {claim}"
    messages = [
        system_prompt,
        HumanMessage(content=text),
    ]
    ai_msg = llm.invoke(messages)
    return ai_msg.content.strip()


df["validation"] = df.apply(lambda x: validate_claim(x["post"], x["gold"]), axis=1)

In [ ]:
disp(df[["post", "gold", "validation"]])

In [ ]:
disp(df[["post", "gold", "validation"]])

In [ ]:
schema = """
{
  "title": "FactCheckResult",
  "type": "object",
  "properties": {
    "is_factually_correct": {
      "title": "Is Factually Correct",
      "type": "boolean",
      "description": "Indicates whether the extracted claim is factually accurate based on the original post.",
      "default": false,
      "examples": [
        true,
        false
      ]
    },
    "errors": {
      "$ref": "#/definitions/ErrorGroups"
    }
  },
  "required": [
    "is_factually_correct",
    "errors"
  ],
  "propertyOrder": [
    "is_factually_correct",
    "errors"
  ],
  "definitions": {
    "ErrorGroups": {
      "title": "ErrorGroups",
      "type": "object",
      "description": "Grouped lists of factual inconsistencies detected in the claim.",
      "propertyOrder": [
        "wrong_name",
        "wrong_number",
        "missing_data",
        "added_data"
      ],
      "properties": {
        "wrong_name": {
          "title": "Wrong Name",
          "type": "array",
          "items": {
            "type": "string"
          },
          "description": "Names, places, or organizations incorrectly stated in the claim.",
          "default": [],
          "examples": [
            [
              "Claim mentions 'Barack Obama' but original post says 'Joe Biden'."
            ]
          ]
        },
        "wrong_number": {
          "title": "Wrong Number",
          "type": "array",
          "items": {
            "type": "string"
          },
          "description": "Numerical inaccuracies in the claim compared to the original post.",
          "default": [],
          "examples": [
            [
              "Claim says '10,000 people' but original says '1,000'."
            ]
          ]
        },
        "missing_data": {
          "title": "Missing Data",
          "type": "array",
          "items": {
            "type": "string"
          },
          "description": "Important context or facts missing from the claim but present in the original.",
          "default": [],
          "examples": [
            [
              "Claim omits that the funds were conditional on approval."
            ]
          ]
        },
        "added_data": {
          "title": "Added Data",
          "type": "array",
          "items": {
            "type": "string"
          },
          "description": "Information added in the claim that was not mentioned in the original post.",
          "default": [],
          "examples": [
            [
              "Claim adds 'military support' while the original only mentions 'infrastructure aid'."
            ]
          ]
        }
      },
      "required": [
        "wrong_name",
        "wrong_number",
        "missing_data",
        "added_data"
      ]
    }
  }
}
"""